<h1>Scraping Fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [5]:
import requests
from bs4 import BeautifulSoup

In [8]:
def get_us_sector_performance():
    output_list = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    output_list = list()
    sector_list = list()
    sector_url_list = list()
    names = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    response = requests.get(url)
    if response.status_code != 200:
        print("Failure")
      
    results_page = BeautifulSoup(response.content,'lxml')
    a = results_page.find_all('div', class_ = "heading")
    b = []
    for i in range(1, len(a)):
        b.append(a[i].find_all('a'))
    flat_list = [item for sublist in b for item in sublist]
    for i in flat_list:
        sector_list.append("https://eresearch.fidelity.com"+i['href'])#links
    list_ = results_page.find_all('strong')#names
    names = []
    for i in list_:
        b = i.get_text()
        names.append(b)
    m = [s for s in names[9: ]] 
    for i in range(0, len(m)):
        empty = []
        sector_data = list(get_sector_change_and_market_cap(sector_list[i]))
        empty.extend([m[i]])
        empty.extend(sector_data)
        empty.append(sector_list[i])
        output_list.append(tuple(empty))
    output_list.sort(reverse=True, key=lambda k: k[3])
    return output_list
    

In [9]:
def get_sector_change_and_market_cap(sector_page_link):
    
    response = requests.get(sector_page_link)    
    
    if response.status_code != 200:
         print("Failure")
    else:      
        results_page = BeautifulSoup(response.content,'lxml')
    
    if results_page.find('span', {'class': 'negative'}):
        sector_change = float(results_page.find('span', {'class': 'negative'}).text.strip('%'))
    else:
        sector_change = float(results_page.find('span', class_ = 'positive').text.strip('%'))
        
    c = results_page.find_all('td')
    d = results_page.find_all('td', class_ = "left-spcng")
    market_weight = c[0].find('span').get_text()
    l = [i.find_all('span') for i in c]
    flat_list2 = [item for sublist in l for item in sublist]
    sector_market_cap = flat_list2[2].get_text()
    sector_market_weight = float(market_weight.strip("%"))
    
    return sector_change,sector_market_cap,sector_market_weight

In [10]:
get_us_sector_performance()

[('Utilities',
  0.96,
  '$1.21T',
  0.96,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Communication Services',
  0.8,
  '$1.75T',
  0.8,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Consumer Discretionary',
  0.54,
  '$5.88T',
  0.54,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Information Technology',
  0.54,
  '$10.00T',
  0.54,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  0.44,
  '$5.80T',
  0.44,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Industrials',
  0.13,
  '$4.30T',
  0.13,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Real Es

In [8]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return (-0.40, $3.58T, 6.80)
#Note that neither the -0.40, nor the 6.80, end with a %sign

(0.54, '$5.88T', 0.54)

In [9]:
#Test get_us_sector_performance()
get_us_sector_performance()
#Should return (example: obviously the results will vary over time!)
"""
[('Telecommunication Services',
  0.21,
  '$1.74T',
  2.0,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Materials',
  0.22,
  '$1.95T',
  2.49,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),
 ('Real Estate',
  -0.45,
  '$1.22T',
  2.7,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
 ('Utilities',
  -0.33,
  '$1.25T',
  2.86,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Energy',
  0.76,
  '$3.90T',
  5.83,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
 ('Consumer Staples',
  -0.32,
  '$3.58T',
  6.8,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
 ('Industrials',
  0.72,
  '$4.31T',
  9.83,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Consumer Discretionary',
  1.03,
  '$5.76T',
  12.9,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Financials',
  0.39,
  '$7.45T',
  13.71,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Health Care',
  0.76,
  '$5.70T',
  14.71,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Information Technology',
  0.81,
  '$9.84T',
  26.17,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45')]
  
"""

"\n[('Telecommunication Services',\n  0.21,\n  '$1.74T',\n  2.0,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),\n ('Materials',\n  0.22,\n  '$1.95T',\n  2.49,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),\n ('Real Estate',\n  -0.45,\n  '$1.22T',\n  2.7,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),\n ('Utilities',\n  -0.33,\n  '$1.25T',\n  2.86,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),\n ('Energy',\n  0.76,\n  '$3.90T',\n  5.83,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),\n ('Consumer Staples',\n  -0.32,\n  '$3.58T',\n  6.8,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'

<h3>Sorting</h3>

In [ ]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort() #Sorts by the first element of the tuple
x

In [ ]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort(key=lambda k: k[1]) #Sorts by the element at position 1
x